# Data Processing Pipeline Demo: From Raw to Analysis-Ready Data

This notebook demonstrates the complete data processing pipeline for Lightcast job market data, showcasing how our custom classes transform raw CSV data into clean, analysis-ready datasets.

## Pipeline Overview

**Raw Data** → **Data Loading** → **Cleaning & Standardization** → **Feature Engineering** → **Quality Assessment** → **Export & Persistence**

## Key Components Demonstrated

- **SparkJobAnalyzer**: Scalable data loading and analysis engine
- **JobMarketDataProcessor**: Comprehensive data cleaning and feature engineering
- **SalaryVisualizer**: Visualization and analysis utilities
- **Multi-format Export**: Parquet, CSV, JSON schema generation
- **Performance Monitoring**: Processing metrics and benchmarks

## Target Audience

- Data engineers implementing similar pipelines
- Developers wanting to understand our data processing approach
- Analysts who need to understand data transformations
- Anyone interested in production-quality data processing with PySpark

## Section 1: Environment Setup and Dependencies

First, we'll set up our environment and import the necessary libraries including our custom classes.

In [1]:
# Environment Setup and Library Imports
import sys
import os
import time
from pathlib import Path
import warnings
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add src directory to path for custom imports
sys.path.append('../src')

print("Setting up Data Processing Pipeline Demo")
print("=" * 50)
print(f"Demo started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Python version: {sys.version.split()[0]}")

# Import our custom classes
try:
    from data.spark_analyzer import SparkJobAnalyzer, create_raw_analyzer
    from data.enhanced_processor import JobMarketDataProcessor
    from data.full_dataset_processor import clean_and_process_data_optimized
    from visualization.simple_plots import SalaryVisualizer
    
    print("\nCustom Classes Imported Successfully:")
    print("  - SparkJobAnalyzer: Data loading and analysis")
    print("  - JobMarketDataProcessor: Data cleaning and processing") 
    print("  - SalaryVisualizer: Visualization utilities")
    print("  - clean_and_process_data_optimized: Advanced processing pipeline")
    
except ImportError as e:
    print(f"Error importing custom classes: {e}")
    print("Please ensure you're running from the notebooks directory")

# Import standard data processing libraries
try:
    import pandas as pd
    import numpy as np
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, count, avg, min as spark_min, max as spark_max
    
    print("\nStandard Libraries Imported:")
    print("  - pandas, numpy: Data manipulation")
    print("  - pyspark: Distributed computing")
    
except ImportError as e:
    print(f"Error importing standard libraries: {e}")

print("\nEnvironment Setup Complete!")
print("Ready to demonstrate the data processing pipeline.")

Setting up Data Processing Pipeline Demo
Demo started at: 2025-09-29 21:37:48
Python version: 3.12.9

Custom Classes Imported Successfully:
  - SparkJobAnalyzer: Data loading and analysis
  - JobMarketDataProcessor: Data cleaning and processing
  - SalaryVisualizer: Visualization utilities
  - clean_and_process_data_optimized: Advanced processing pipeline

Standard Libraries Imported:
  - pandas, numpy: Data manipulation
  - pyspark: Distributed computing

Environment Setup Complete!
Ready to demonstrate the data processing pipeline.

Custom Classes Imported Successfully:
  - SparkJobAnalyzer: Data loading and analysis
  - JobMarketDataProcessor: Data cleaning and processing
  - SalaryVisualizer: Visualization utilities
  - clean_and_process_data_optimized: Advanced processing pipeline

Standard Libraries Imported:
  - pandas, numpy: Data manipulation
  - pyspark: Distributed computing

Environment Setup Complete!
Ready to demonstrate the data processing pipeline.


In [2]:
# Verify Data File Availability and Configure Spark
print("Checking Data File Availability")
print("=" * 40)

# Define expected data sources
data_sources = {
    "raw_lightcast": "../data/raw/lightcast_job_postings.csv",
    "processed_parquet": "../data/processed/job_market_processed.parquet",
    "sample_csv": "../data/processed/clean_job_data.csv"
}

# Check file availability
available_files = {}
for name, path in data_sources.items():
    file_path = Path(path)
    exists = file_path.exists()
    status = "Available" if exists else "Missing"
    
    print(f"{name:20}: {status}")
    
    if exists:
        if path.endswith('.csv'):
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"{'':20}  Size: {size_mb:.1f} MB")
        available_files[name] = path

print(f"\nFiles found: {len(available_files)}/{len(data_sources)}")

# Verify we have the raw data file (minimum requirement)
if "raw_lightcast" not in available_files:
    print("\nWARNING: Raw Lightcast data file not found!")
    print("Please ensure '../data/raw/lightcast_job_postings.csv' exists")
    print("This demo requires the raw data file to demonstrate the full pipeline.")
else:
    print("\nAll required files found - ready to proceed with demo!")

# Initialize Spark configuration for demo
print("\nConfiguring Spark Session for Demo")
print("-" * 35)

spark_config = {
    "spark.app.name": "DataProcessingPipelineDemo",
    "spark.sql.adaptive.enabled": "true",
    "spark.sql.adaptive.coalescePartitions.enabled": "true",
    "spark.driver.memory": "4g",
    "spark.driver.maxResultSize": "2g"
}

print("Spark Configuration:")
for key, value in spark_config.items():
    print(f"  {key}: {value}")

print("\nEnvironment verification complete!")

Checking Data File Availability
raw_lightcast       : Available
                      Size: 683.5 MB
processed_parquet   : Missing
sample_csv          : Missing

Files found: 1/3

All required files found - ready to proceed with demo!

Configuring Spark Session for Demo
-----------------------------------
Spark Configuration:
  spark.app.name: DataProcessingPipelineDemo
  spark.sql.adaptive.enabled: true
  spark.sql.adaptive.coalescePartitions.enabled: true
  spark.driver.memory: 4g
  spark.driver.maxResultSize: 2g

Environment verification complete!


## Section 2: Raw Data Loading and Validation

Now we'll demonstrate how to load raw Lightcast data using our SparkJobAnalyzer class and perform initial data quality checks.

In [3]:
# Demonstrate Raw Data Loading with SparkJobAnalyzer
print("STAGE 1: RAW DATA LOADING")
print("=" * 50)

# Method 1: Using create_raw_analyzer() - Forces fresh raw data load
print("Loading raw data using create_raw_analyzer()...")
start_time = time.time()

try:
    # This function forces loading from raw CSV, bypassing any processed data
    analyzer = create_raw_analyzer()
    
    load_time = time.time() - start_time
    print(f"Raw data loaded successfully in {load_time:.2f} seconds")
    
    # Get basic dataset metrics
    raw_df = analyzer.get_df()
    record_count = raw_df.count()
    column_count = len(raw_df.columns)
    
    print(f"\nDataset Overview:")
    print(f"  Total Records: {record_count:,}")
    print(f"  Total Columns: {column_count}")
    print(f"  Loading Method: SparkJobAnalyzer.create_raw_analyzer()")
    
    # Show data types summary
    print(f"\nColumn Type Distribution:")
    schema_summary = {}
    for field in raw_df.schema.fields:
        field_type = str(field.dataType)
        schema_summary[field_type] = schema_summary.get(field_type, 0) + 1
    
    for dtype, count in sorted(schema_summary.items()):
        print(f"  {dtype}: {count} columns")
        
except Exception as e:
    print(f"Error loading raw data: {e}")
    analyzer = None

STAGE 1: RAW DATA LOADING
Loading raw data using create_raw_analyzer()...


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/29 21:37:50 WARN Utils: Your hostname, SamWin, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/09/29 21:37:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/29 21:37:50 WARN Utils: Your hostname, SamWin, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/09/29 21:37:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newL

Raw data loaded successfully in 13.46 seconds



Dataset Overview:
  Total Records: 72,498
  Total Columns: 131
  Loading Method: SparkJobAnalyzer.create_raw_analyzer()

Column Type Distribution:
  BooleanType(): 2 columns
  IntegerType(): 38 columns
  StringType(): 90 columns
  TimestampType(): 1 columns


In [4]:
# Perform Initial Data Quality Assessment
if analyzer is not None:
    print("\nSTAGE 2: INITIAL DATA QUALITY ASSESSMENT")
    print("=" * 50)
    
    raw_df = analyzer.job_data
    
    # 1. Display sample records with key columns
    print("Sample Data (First 3 records, key columns):")
    key_columns = ['TITLE', 'COMPANY', 'LOCATION', 'SALARY_AVG_IMPUTED']
    existing_key_cols = [col for col in key_columns if col in raw_df.columns]
    
    if existing_key_cols:
        raw_df.select(existing_key_cols).show(3, truncate=True)
    
    # 2. Column categorization for better understanding
    all_columns = raw_df.columns
    print(f"\nColumn Organization ({len(all_columns)} total):")
    
    column_categories = {
        'Identity': [col for col in all_columns if any(x in col.upper() for x in ['ID', 'JOB_ID'])],
        'Basic Info': [col for col in all_columns if any(x in col.upper() for x in ['TITLE', 'COMPANY', 'DESCRIPTION'])],
        'Location': [col for col in all_columns if any(x in col.upper() for x in ['LOCATION', 'CITY', 'STATE', 'COUNTRY'])],
        'Salary': [col for col in all_columns if 'SALARY' in col.upper()],
        'Employment': [col for col in all_columns if any(x in col.upper() for x in ['EMPLOYMENT', 'EXPERIENCE', 'EDUCATION'])],
        'Industry': [col for col in all_columns if 'INDUSTRY' in col.upper()],
        'Remote/AI': [col for col in all_columns if any(x in col.upper() for x in ['REMOTE', 'AI'])],
        'Temporal': [col for col in all_columns if any(x in col.upper() for x in ['DATE', 'TIME', 'POSTED'])],
    }
    
    for category, cols in column_categories.items():
        if cols:
            print(f"  {category}: {len(cols)} columns")
            if len(cols) <= 5:  # Show column names if not too many
                print(f"    {', '.join(cols)}")
    
    # 3. Quick null analysis for critical columns
    print(f"\nNull Value Analysis (Critical Columns):")
    critical_columns = ['TITLE', 'COMPANY', 'LOCATION', 'SALARY_AVG_IMPUTED']
    existing_critical = [col for col in critical_columns if col in raw_df.columns]
    
    for col in existing_critical:
        null_count = raw_df.filter(raw_df[col].isNull()).count()
        null_pct = (null_count / record_count) * 100 if record_count > 0 else 0
        print(f"  {col}: {null_count:,} nulls ({null_pct:.1f}%)")
    
    print("\nRaw data loading and initial assessment complete!")
    print("Data is ready for the cleaning and processing pipeline.")
else:
    print("Cannot proceed with quality assessment - raw data not loaded")


STAGE 2: INITIAL DATA QUALITY ASSESSMENT
Sample Data (First 3 records, key columns):
+------------------+--------+--------------------+
|             TITLE| COMPANY|            LOCATION|
+------------------+--------+--------------------+
|ET29C073C03D1F86B4|  894731|{\n  "lat": 33.20...|
|ET21DDA63780A7DC09|  133098|{\n  "lat": 44.31...|
|ET3037E0C947A02404|39063746|{\n  "lat": 32.77...|
+------------------+--------+--------------------+
only showing top 3 rows

Column Organization (131 total):
  Identity: 1 columns
    ID
  Basic Info: 8 columns
  Location: 5 columns
    LOCATION, CITY, CITY_NAME, STATE, STATE_NAME
  Salary: 3 columns
    SALARY, SALARY_TO, SALARY_FROM
  Employment: 6 columns
  Remote/AI: 22 columns
  Temporal: 3 columns
    LAST_UPDATED_DATE, LAST_UPDATED_TIMESTAMP, POSTED

Null Value Analysis (Critical Columns):
+------------------+--------+--------------------+
|             TITLE| COMPANY|            LOCATION|
+------------------+--------+--------------------+
|E

  TITLE: 44 nulls (0.1%)


  COMPANY: 44 nulls (0.1%)


  LOCATION: 44 nulls (0.1%)

Raw data loading and initial assessment complete!
Data is ready for the cleaning and processing pipeline.


## Section 3: Data Cleaning Pipeline Demonstration

This section showcases our JobMarketDataProcessor class and its comprehensive data cleaning capabilities.

In [22]:
# Initialize JobMarketDataProcessor and Demonstrate Cleaning Pipeline
if analyzer is not None:
    print("STAGE 3: DATA CLEANING PIPELINE")
    print("=" * 50)
    
    # Initialize our data processor
    print("Initializing JobMarketDataProcessor...")
    processor = JobMarketDataProcessor("PipelineDemo")
    
    # Set the raw data
    processor.df_raw = analyzer.job_data
    print(f"Processor initialized with {processor.df_raw.count():,} raw records")
    
    # Demonstrate the cleaning pipeline step by step
    print("\nApplying Data Cleaning Pipeline...")
    
    # Record initial state
    initial_count = processor.df_raw.count()
    initial_columns = set(processor.df_raw.columns)
    
    start_time = time.time()
    
    try:
        # Apply the cleaning and standardization
        print("Calling clean_and_process_data_optimized()...")
        cleaned_df = processor.clean_and_process_data_optimized(processor.df_raw)
        
        cleaning_time = time.time() - start_time
        
        print(f"Method completed. Returned DataFrame: {cleaned_df is not None}")
        
        # Check if the processor has stored the processed data
        if hasattr(processor, 'df_processed'):
            print(f"processor.df_processed exists: {processor.df_processed is not None}")
            if processor.df_processed is not None:
                print("✓ Processor df_processed is set correctly")
            else:
                print("! Setting processor.df_processed manually")
                processor.df_processed = cleaned_df
        else:
            print("! processor.df_processed attribute does not exist, creating it")
            processor.df_processed = cleaned_df
        
        # Verify the cleaned data
        if cleaned_df is not None:
            # Analyze the results
            final_count = cleaned_df.count()
            final_columns = set(cleaned_df.columns)
            
            # Calculate changes
            record_change = final_count - initial_count
            new_columns = final_columns - initial_columns
            removed_columns = initial_columns - final_columns
            
            print(f"\nCleaning Pipeline Results:")
            print(f"  Processing Time: {cleaning_time:.2f} seconds")
            print(f"  Records Before: {initial_count:,}")
            print(f"  Records After:  {final_count:,}")
            print(f"  Record Change:  {record_change:+,}")
            
            if record_change != 0:
                pct_change = (record_change / initial_count) * 100
                print(f"  Percentage Change: {pct_change:+.2f}%")
            
            print(f"\nColumn Changes:")
            print(f"  Columns Before: {len(initial_columns)}")
            print(f"  Columns After:  {len(final_columns)}")
            
            if new_columns:
                print(f"  New Columns Created ({len(new_columns)}):")
                for col in sorted(list(new_columns)[:5]):  # Show first 5
                    print(f"    + {col}")
                if len(new_columns) > 5:
                    print(f"    ... and {len(new_columns) - 5} more")
            
            if removed_columns:
                print(f"  Columns Removed ({len(removed_columns)}):")
                for col in sorted(list(removed_columns)[:3]):  # Show first 3
                    print(f"    - {col}")
                if len(removed_columns) > 3:
                    print(f"    ... and {len(removed_columns) - 3} more")
            
            print("\nData cleaning pipeline completed successfully!")
            print("✓ Processed data is ready for next stages")
        else:
            print("ERROR: Cleaning method returned None")
        
    except Exception as e:
        import traceback
        print(f"Error in cleaning pipeline: {e}")
        print("Full traceback:")
        traceback.print_exc()
        cleaned_df = None
        
else:
    print("Cannot proceed with cleaning - raw data not available")

INFO:data.enhanced_processor:Initializing Enhanced Job Market Data Processor...
INFO:data.enhanced_processor:Spark session initialized: 4.0.1
INFO:data.enhanced_processor:Spark session initialized: 4.0.1
INFO:data.enhanced_processor:Schema defined with 21 fields
INFO:data.enhanced_processor:Schema defined with 21 fields


STAGE 3: DATA CLEANING PIPELINE
Initializing JobMarketDataProcessor...


Processor initialized with 72,498 raw records

Applying Data Cleaning Pipeline...


INFO:data.enhanced_processor:=== STARTING OPTIMIZED DATA CLEANING PIPELINE ===  
INFO:data.enhanced_processor:=== STARTING OPTIMIZED DATA CLEANING PIPELINE ===  


Calling clean_and_process_data_optimized()...


INFO:data.enhanced_processor:BEFORE CLEANING:                                   
INFO:data.enhanced_processor:   → Columns: 131
INFO:data.enhanced_processor:   → Records: 72,498
INFO:data.enhanced_processor:Step 1: Dropping non-essential columns...
INFO:data.enhanced_processor:   ✅ Dropped columns: ['LAST_UPDATED_DATE', 'LAST_UPDATED_TIMESTAMP', 'ACTIVE_SOURCES_INFO']
INFO:data.enhanced_processor:Step 2: Handling REMOTE_TYPE_NAME nulls...
INFO:data.enhanced_processor:BEFORE CLEANING:                                   
INFO:data.enhanced_processor:   → Columns: 131
INFO:data.enhanced_processor:   → Records: 72,498
INFO:data.enhanced_processor:Step 1: Dropping non-essential columns...
INFO:data.enhanced_processor:   ✅ Dropped columns: ['LAST_UPDATED_DATE', 'LAST_UPDATED_TIMESTAMP', 'ACTIVE_SOURCES_INFO']
INFO:data.enhanced_processor:Step 2: Handling REMOTE_TYPE_NAME nulls...
INFO:data.enhanced_processor:   → REMOTE_TYPE_NAME nulls: 44 (0.1%)             
INFO:data.enhanced_processor:   →

Error in cleaning pipeline: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `SALARY_MIN` cannot be resolved. Did you mean one of the following? [`SALARY_TO`, `SALARY`, `SALARY_FROM`, `MSA_NAME`, `BODY`]. SQLSTATE: 42703;
'Project [ID#0, DUPLICATES#3, POSTED#4, EXPIRED#5, DURATION#6, SOURCE_TYPES#7, SOURCES#8, URL#9, ACTIVE_URLS#10, TITLE_RAW#12, BODY#13, MODELED_EXPIRED#14, MODELED_DURATION#15, COMPANY#16, COMPANY_NAME#17, COMPANY_RAW#18, COMPANY_IS_STAFFING#19, EDUCATION_LEVELS#20, EDUCATION_LEVELS_NAME#21, MIN_EDULEVELS#22, MIN_EDULEVELS_NAME#23, MAX_EDULEVELS#24, MAX_EDULEVELS_NAME#25, EMPLOYMENT_TYPE#26, EMPLOYMENT_TYPE_NAME#27, MIN_YEARS_EXPERIENCE#28, MAX_YEARS_EXPERIENCE#29, IS_INTERNSHIP#30, SALARY#31, REMOTE_TYPE#32, REMOTE_TYPE_NAME#6074, ORIGINAL_PAY_PERIOD#34, SALARY_TO#35, SALARY_FROM#36, LOCATION#37, COUNTY#40, COUNTY_NAME#41, MSA#42, MSA_NAME#43, STATE#44, STATE_NAME#45, COUNTY_ID#6092, COUNTY_NAME#6107, MSA_OUTGOING#50, MSA_NAME_O

Traceback (most recent call last):
  File "/tmp/ipykernel_4268/814263455.py", line 26, in <module>
    cleaned_df = processor.clean_and_process_data_optimized(processor.df_raw)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/samarthya/sourcebox/github.com/project-from-scratch/notebooks/../src/data/enhanced_processor.py", line 788, in clean_and_process_data_optimized
    df_cleaned = self.clean_and_process_data(df_cleaned)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/samarthya/sourcebox/github.com/project-from-scratch/notebooks/../src/data/enhanced_processor.py", line 452, in clean_and_process_data
    df_clean = df_clean.withColumn("SALARY_MIN_CLEAN", when(col("SALARY_MIN").rlike("^[0-9]+$"), col("SALARY_MIN").cast("double")).otherwise(None)).withColumn("SALARY_MAX_CLEAN", when(col("SALARY_MAX").rlike("^[0-9]+$"), col("SALARY_MAX").cast("double")).otherwise(None)).withColumn("SALARY_AVG", (col("SALARY_MIN_CLEAN") + c

In [6]:
# Demonstrate Before/After Comparison of Key Data Quality Improvements
if 'cleaned_df' in locals() and cleaned_df is not None:
    print("\nSTAGE 4: BEFORE/AFTER DATA QUALITY COMPARISON")
    print("=" * 55)
    
    # Compare company name standardization (null handling)
    print("1. Company Name Standardization:")
    
    # Before: Check null companies in raw data
    raw_null_companies = processor.df_raw.filter(col("COMPANY").isNull()).count()
    
    # After: Check null companies in cleaned data  
    clean_null_companies = cleaned_df.filter(col("COMPANY").isNull()).count()
    
    print(f"   Raw Data - Null Companies: {raw_null_companies:,}")
    print(f"   Cleaned Data - Null Companies: {clean_null_companies:,}")
    print(f"   Improvement: {raw_null_companies - clean_null_companies:,} nulls handled")
    
    # Show sample of company standardization
    print("\n   Sample Company Name Standardization:")
    if "COMPANY" in cleaned_df.columns:
        company_sample = cleaned_df.select("COMPANY").distinct().limit(5).collect()
        for row in company_sample:
            print(f"   '{row.COMPANY}'")
    
    # 2. Location data standardization
    if "LOCATION" in cleaned_df.columns:
        print("\n2. Location Data Standardization:")
        location_sample = cleaned_df.select("LOCATION").filter(col("LOCATION").isNotNull()).limit(3).collect()
        for row in location_sample:
            print(f"   '{row.LOCATION}'")
    
    # 3. Salary data validation
    if any("SALARY" in col_name for col_name in cleaned_df.columns):
        print("\n3. Salary Data Quality:")
        salary_cols = [col_name for col_name in cleaned_df.columns if "SALARY" in col_name]
        
        for sal_col in salary_cols[:2]:  # Show first 2 salary columns
            non_null_count = cleaned_df.filter(col(sal_col).isNotNull()).count()
            coverage = (non_null_count / final_count) * 100 if final_count > 0 else 0
            print(f"   {sal_col}: {non_null_count:,} values ({coverage:.1f}% coverage)")
    
    print("\nData quality improvements applied successfully!")
    print("Ready for feature engineering stage.")
    
else:
    print("Cannot show before/after comparison - cleaned data not available")

Cannot show before/after comparison - cleaned data not available


## Section 4: Feature Engineering Process

Demonstrate the automated feature engineering capabilities that create derived columns and enhance the dataset for analysis.

In [26]:
# Feature Engineering Analysis - Working Version
print("DEBUG: Checking processor and data state...")

# Check available data sources and force assignment
if cleaned_df is not None and processor is not None:
    print("✓ Found cleaned_df and processor - setting up analysis...")
    processor.df_processed = cleaned_df
    
    print("STAGE 5: FEATURE ENGINEERING ANALYSIS")
    print("=" * 50)
    
    print("Analyzing feature engineering results from cleaning pipeline...")
    print("(Feature engineering was automatically applied during data cleaning)")
    
    enhanced_df = processor.df_processed
    
    # Analyze the feature engineering results that were applied during cleaning
    all_columns = set(enhanced_df.columns)
    
    print(f"\nDataset Analysis:")
    print(f"  Total Columns: {len(all_columns)}")
    print(f"  Total Records: {enhanced_df.count():,}")
    
    # Show a sample of column names to understand what was created
    print(f"\nSample Column Names:")
    sample_cols = sorted(list(all_columns))[:15]
    for i, col_name in enumerate(sample_cols):
        marker = "  •"
        if "_CLEAN" in col_name:
            marker = "  ✓"
        elif "_IMPUTED" in col_name:
            marker = "  +"
        elif "_AVG" in col_name:
            marker = "  $"
        print(f"{marker} {col_name}")
    
    if len(all_columns) > 15:
        print(f"  ... and {len(all_columns) - 15} more columns")
    
    # Categorize features
    feature_categories = {
        'Original Fields': [f for f in all_columns if not any(suffix in f for suffix in ['_CLEAN', '_IMPUTED', '_AVG'])],
        'Cleaned Fields': [f for f in all_columns if f.endswith('_CLEAN')],
        'Imputed Fields': [f for f in all_columns if '_IMPUTED' in f],
        'Calculated Fields': [f for f in all_columns if '_AVG' in f]
    }
    
    print(f"\nFeature Category Summary:")
    for category, features in feature_categories.items():
        print(f"  {category}: {len(features)} columns")
    
    # Show sample records for key columns
    key_columns = [col_name for col_name in ['TITLE', 'COMPANY', 'SALARY_AVG_IMPUTED', 'COMPANY_CLEAN'] if col_name in all_columns]
    if key_columns:
        print(f"\nSample Data (first 2 records, key columns):")
        try:
            sample_data = enhanced_df.select(key_columns).limit(2).collect()
            for i, row in enumerate(sample_data, 1):
                print(f"  Record {i}:")
                for col_name in key_columns:
                    value = getattr(row, col_name, 'N/A')
                    if value is None:
                        value = 'NULL'
                    elif isinstance(value, str) and len(value) > 40:
                        value = value[:37] + "..."
                    elif isinstance(value, (int, float)):
                        if 'SALARY' in col_name:
                            value = f"${value:,.0f}"
                    print(f"    {col_name}: {value}")
        except Exception as e:
            print(f"    (Sample data display failed: {e})")
    
    print(f"\nFeature engineering analysis completed!")
    print(f"Dataset is analysis-ready with {len(all_columns)} total columns.")
    print(f"✓ Processor.df_processed is now properly set")
    
else:
    print("ERROR: Cannot find necessary data")
    print(f"cleaned_df is None: {cleaned_df is None}")
    print(f"processor is None: {processor is None}")

DEBUG: Checking processor and data state...
ERROR: Cannot find necessary data
cleaned_df is None: True
processor is None: False


## Section 5: Data Quality Assessment

Perform comprehensive quality checks on the fully processed dataset to ensure it meets our standards for analysis.

In [27]:
# Comprehensive Data Quality Assessment on Processed Dataset
if hasattr(processor, 'df_processed') and processor.df_processed is not None:
    print("STAGE 6: COMPREHENSIVE DATA QUALITY ASSESSMENT")
    print("=" * 60)
    
    final_df = processor.df_processed
    
    # Create analyzer for the processed data
    processed_analyzer = SparkJobAnalyzer()
    processed_analyzer.job_data = final_df
    processed_analyzer.job_data.createOrReplaceTempView("processed_jobs")
    
    print("1. Dataset Overview:")
    print(f"   Final Record Count: {final_df.count():,}")
    print(f"   Final Column Count: {len(final_df.columns)}")
    
    # Generate comprehensive statistics using our analyzer
    try:
        stats = processed_analyzer.get_overall_statistics()
        
        print(f"\n2. Key Statistics:")
        metrics_to_show = ['total_records', 'unique_companies', 'unique_locations', 'median_salary']
        
        for metric in metrics_to_show:
            if metric in stats:
                value = stats[metric]
                if 'salary' in metric and isinstance(value, (int, float)):
                    print(f"   {metric.replace('_', ' ').title()}: ${value:,.0f}")
                else:
                    print(f"   {metric.replace('_', ' ').title()}: {value:,}")
                    
    except Exception as e:
        print(f"   Could not generate statistics: {e}")
    
    # 3. Data completeness analysis for critical columns
    print(f"\n3. Data Completeness Analysis:")
    critical_analysis_columns = ['TITLE', 'COMPANY', 'SALARY_AVG_IMPUTED', 'EXPERIENCE_LEVEL_CLEAN']
    existing_analysis_cols = [col for col in critical_analysis_columns if col in final_df.columns]
    
    total_records = final_df.count()
    
    for col in existing_analysis_cols:
        non_null_count = final_df.filter(col(col).isNotNull()).count()
        completeness = (non_null_count / total_records) * 100 if total_records > 0 else 0
        print(f"   {col}: {completeness:.1f}% complete ({non_null_count:,}/{total_records:,})")
    
    # 4. Business rule validation
    print(f"\n4. Business Rule Validation:")
    
    # Check salary ranges
    if 'SALARY_AVG_IMPUTED' in final_df.columns:
        salary_stats = final_df.select(
            spark_min('SALARY_AVG_IMPUTED').alias('min_salary'),
            spark_max('SALARY_AVG_IMPUTED').alias('max_salary'),
            avg('SALARY_AVG_IMPUTED').alias('avg_salary')
        ).collect()[0]
        
        print(f"   Salary Range: ${salary_stats.min_salary:,.0f} - ${salary_stats.max_salary:,.0f}")
        print(f"   Average Salary: ${salary_stats.avg_salary:,.0f}")
        
        # Validate reasonable salary ranges
        reasonable_salaries = final_df.filter(
            (col('SALARY_AVG_IMPUTED') >= 20000) & 
            (col('SALARY_AVG_IMPUTED') <= 500000)
        ).count()
        reasonable_pct = (reasonable_salaries / total_records) * 100 if total_records > 0 else 0
        print(f"   Reasonable Salaries: {reasonable_pct:.1f}% ({reasonable_salaries:,} records)")
    
    # Check for duplicate records (if ID column exists)
    id_columns = [col for col in final_df.columns if 'ID' in col.upper()]
    if id_columns:
        id_col = id_columns[0]
        unique_ids = final_df.select(id_col).distinct().count()
        duplicate_rate = ((total_records - unique_ids) / total_records) * 100 if total_records > 0 else 0
        print(f"   Duplicate Rate: {duplicate_rate:.2f}% (based on {id_col})")
    
    print(f"\n5. Data Distribution Summary:")
    # Show top companies, locations, etc.
    if 'COMPANY' in final_df.columns:
        top_companies = final_df.groupBy('COMPANY').count().orderBy(col('count').desc()).limit(3).collect()
        print(f"   Top Companies:")
        for row in top_companies:
            print(f"     {row.COMPANY}: {row.count:,} jobs")
    
    print("\nData quality assessment completed!")
    print("Dataset has passed quality checks and is ready for analysis and export.")
    
else:
    print("Cannot perform quality assessment - processed data not available")

Cannot perform quality assessment - processed data not available


## Section 6: Export and Persistence

Now that we have processed, validated data, we'll demonstrate how to export it in multiple formats and validate our persistence operations.

In [9]:
# Export and Persistence Operations
import os
from datetime import datetime

if hasattr(processor, 'df_processed') and processor.df_processed is not None:
    print("SECTION 6: EXPORT AND PERSISTENCE")
    print("=" * 50)
    
    processed_data = processor.df_processed
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Define export paths
    export_base_path = "/home/samarthya/sourcebox/github.com/project-from-scratch/data/processed"
    parquet_path = f"{export_base_path}/processed_jobs_{timestamp}.parquet"
    csv_path = f"{export_base_path}/processed_jobs_{timestamp}.csv"
    
    # Ensure directory exists
    os.makedirs(export_base_path, exist_ok=True)
    
    print("1. Multi-Format Export Operations:")
    
    # Export to Parquet (efficient for analytics)
    try:
        print("   Exporting to Parquet format...")
        processed_data.coalesce(1).write.mode("overwrite").parquet(parquet_path)
        print(f"   ✓ Parquet export completed: {parquet_path}")
        
        # Verify parquet file
        parquet_verification = spark.read.parquet(parquet_path)
        parquet_count = parquet_verification.count()
        print(f"   ✓ Parquet verification: {parquet_count:,} records")
        
    except Exception as e:
        print(f"   ✗ Parquet export failed: {e}")
    
    # Export to CSV (for external tools)
    try:
        print("   Exporting to CSV format...")
        processed_data.coalesce(1).write.mode("overwrite") \
            .option("header", "true") \
            .option("escape", '"') \
            .csv(csv_path)
        print(f"   ✓ CSV export completed: {csv_path}")
        
        # Verify CSV file
        csv_verification = spark.read.option("header", "true").csv(csv_path)
        csv_count = csv_verification.count()
        print(f"   ✓ CSV verification: {csv_count:,} records")
        
    except Exception as e:
        print(f"   ✗ CSV export failed: {e}")
    
    print(f"\n2. Schema and Metadata Export:")
    
    # Export schema information
    try:
        schema_info = {
            'timestamp': timestamp,
            'record_count': processed_data.count(),
            'column_count': len(processed_data.columns),
            'columns': [{'name': field.name, 'type': str(field.dataType)} for field in processed_data.schema.fields],
            'export_paths': {
                'parquet': parquet_path,
                'csv': csv_path
            }
        }
        
        print(f"   Schema captured: {len(schema_info['columns'])} columns")
        print(f"   Record count: {schema_info['record_count']:,}")
        
        # Show sample of column information
        print("   Column Types (first 5):")
        for col_info in schema_info['columns'][:5]:
            print(f"     {col_info['name']}: {col_info['type']}")
        
        # Save schema as JSON
        import json
        schema_path = f"{export_base_path}/schema_{timestamp}.json"
        with open(schema_path, 'w') as f:
            json.dump(schema_info, f, indent=2)
        print(f"   ✓ Schema metadata saved: {schema_path}")
        
    except Exception as e:
        print(f"   ✗ Schema export failed: {e}")
    
    print(f"\n3. Data Loading Validation:")
    
    # Test loading from each export format
    try:
        # Test Parquet loading
        reloaded_parquet = spark.read.parquet(parquet_path)
        parquet_reload_count = reloaded_parquet.count()
        parquet_reload_cols = len(reloaded_parquet.columns)
        
        print(f"   Parquet Reload Test:")
        print(f"     Records: {parquet_reload_count:,}")
        print(f"     Columns: {parquet_reload_cols}")
        
        # Test CSV loading
        reloaded_csv = spark.read.option("header", "true").option("inferSchema", "true").csv(csv_path)
        csv_reload_count = reloaded_csv.count()
        csv_reload_cols = len(reloaded_csv.columns)
        
        print(f"   CSV Reload Test:")
        print(f"     Records: {csv_reload_count:,}")
        print(f"     Columns: {csv_reload_cols}")
        
        # Verify data integrity
        original_count = processed_data.count()
        data_integrity_check = (parquet_reload_count == original_count and csv_reload_count == original_count)
        
        print(f"\n   Data Integrity Check: {'✓ PASSED' if data_integrity_check else '✗ FAILED'}")
        print(f"   Original: {original_count:,} | Parquet: {parquet_reload_count:,} | CSV: {csv_reload_count:,}")
        
    except Exception as e:
        print(f"   ✗ Data loading validation failed: {e}")
    
    print(f"\n4. Export Summary:")
    try:
        # Get file sizes
        import glob
        
        parquet_files = glob.glob(f"{parquet_path}/*.parquet")
        csv_files = glob.glob(f"{csv_path}/*.csv")
        
        if parquet_files:
            parquet_size = sum(os.path.getsize(f) for f in parquet_files)
            print(f"   Parquet Size: {parquet_size / (1024*1024):.2f} MB")
        
        if csv_files:
            csv_size = sum(os.path.getsize(f) for f in csv_files)
            print(f"   CSV Size: {csv_size / (1024*1024):.2f} MB")
        
        print(f"   Export completed successfully!")
        print(f"   Files available in: {export_base_path}")
        
    except Exception as e:
        print(f"   Could not determine file sizes: {e}")
    
else:
    print("Cannot export data - processed data not available")

Cannot export data - processed data not available


## Section 7: Pipeline Performance Metrics

Let's analyze the performance characteristics of our data processing pipeline and measure the efficiency of each stage.

In [10]:
# Pipeline Performance Analysis
import time
import psutil
import gc
from datetime import datetime

print("SECTION 7: PIPELINE PERFORMANCE METRICS")
print("=" * 55)

# Performance metrics collection
performance_metrics = {}

print("1. Memory Usage Analysis:")
try:
    # Get current memory usage
    process = psutil.Process()
    memory_info = process.memory_info()
    memory_mb = memory_info.rss / (1024 * 1024)
    
    print(f"   Current Memory Usage: {memory_mb:.2f} MB")
    
    # Get Spark context memory info if available
    if spark:
        spark_conf = spark.sparkContext.getConf()
        driver_memory = spark_conf.get('spark.driver.memory', 'Not configured')
        executor_memory = spark_conf.get('spark.executor.memory', 'Not configured')
        
        print(f"   Spark Driver Memory: {driver_memory}")
        print(f"   Spark Executor Memory: {executor_memory}")
        
        # Check active Spark jobs
        spark_context = spark.sparkContext
        print(f"   Active Spark Jobs: {len(spark_context.statusTracker().getActiveJobIds())}")
        
    performance_metrics['memory_usage_mb'] = memory_mb
    
except Exception as e:
    print(f"   Could not analyze memory usage: {e}")

print(f"\n2. Data Processing Stages Summary:")

# Calculate processing efficiency if we have stage information
if hasattr(processor, 'df_processed') and processor.df_processed is not None:
    try:
        original_count = raw_analyzer.job_data.count() if hasattr(raw_analyzer, 'job_data') else 0
        processed_count = processor.df_processed.count()
        
        data_retention_rate = (processed_count / original_count * 100) if original_count > 0 else 0
        records_filtered = original_count - processed_count
        
        print(f"   Original Records: {original_count:,}")
        print(f"   Processed Records: {processed_count:,}")
        print(f"   Records Filtered: {records_filtered:,}")
        print(f"   Data Retention Rate: {data_retention_rate:.1f}%")
        
        performance_metrics.update({
            'original_records': original_count,
            'processed_records': processed_count,
            'data_retention_rate': data_retention_rate
        })
        
    except Exception as e:
        print(f"   Could not calculate processing metrics: {e}")

print(f"\n3. Feature Engineering Impact:")

# Analyze column creation and transformations
if hasattr(processor, 'df_processed') and processor.df_processed is not None:
    try:
        original_columns = len(raw_analyzer.job_data.columns) if hasattr(raw_analyzer, 'job_data') else 0
        processed_columns = len(processor.df_processed.columns)
        columns_added = processed_columns - original_columns
        
        print(f"   Original Columns: {original_columns}")
        print(f"   Final Columns: {processed_columns}")
        print(f"   Columns Added: {columns_added}")
        print(f"   Feature Enhancement: {(columns_added/original_columns*100):.1f}% increase" if original_columns > 0 else "")
        
        performance_metrics.update({
            'original_columns': original_columns,
            'final_columns': processed_columns,
            'columns_added': columns_added
        })
        
    except Exception as e:
        print(f"   Could not analyze feature engineering: {e}")

print(f"\n4. Processing Time Estimation:")

# Simulate a small processing operation to estimate timing
try:
    if hasattr(processor, 'df_processed') and processor.df_processed is not None:
        print("   Running performance benchmark on sample data...")
        
        # Time a simple aggregation operation
        start_time = time.time()
        sample_agg = processor.df_processed.groupBy('COMPANY').count().collect()
        end_time = time.time()
        
        operation_time = end_time - start_time
        records_per_second = processed_count / operation_time if operation_time > 0 else 0
        
        print(f"   Sample Aggregation Time: {operation_time:.3f} seconds")
        print(f"   Processing Rate: {records_per_second:,.0f} records/second")
        print(f"   Unique Companies Found: {len(sample_agg):,}")
        
        performance_metrics.update({
            'sample_operation_time': operation_time,
            'processing_rate_rps': records_per_second
        })
        
except Exception as e:
    print(f"   Could not run performance benchmark: {e}")

print(f"\n5. Resource Utilization Summary:")

try:
    # CPU usage
    cpu_percent = psutil.cpu_percent(interval=1)
    
    # Disk usage for workspace
    workspace_path = "/home/samarthya/sourcebox/github.com/project-from-scratch"
    disk_usage = psutil.disk_usage(workspace_path)
    
    print(f"   CPU Usage: {cpu_percent:.1f}%")
    print(f"   Workspace Disk Usage: {(disk_usage.used / disk_usage.total * 100):.1f}%")
    print(f"   Available Disk Space: {(disk_usage.free / (1024**3)):.1f} GB")
    
    performance_metrics.update({
        'cpu_percent': cpu_percent,
        'disk_usage_percent': disk_usage.used / disk_usage.total * 100
    })
    
except Exception as e:
    print(f"   Could not analyze resource utilization: {e}")

print(f"\n6. Pipeline Efficiency Assessment:")

try:
    # Calculate overall efficiency score based on available metrics
    efficiency_factors = []
    
    if 'data_retention_rate' in performance_metrics:
        # Higher retention rate is generally better (less data loss)
        retention_score = min(performance_metrics['data_retention_rate'] / 90.0, 1.0)  # 90% as target
        efficiency_factors.append(('Data Retention', retention_score, f"{performance_metrics['data_retention_rate']:.1f}%"))
    
    if 'processing_rate_rps' in performance_metrics:
        # Normalize processing rate (assume 1000 rps as good target)
        rate_score = min(performance_metrics['processing_rate_rps'] / 1000.0, 1.0)
        efficiency_factors.append(('Processing Speed', rate_score, f"{performance_metrics['processing_rate_rps']:,.0f} rps"))
    
    if 'memory_usage_mb' in performance_metrics:
        # Lower memory usage is better (assume 2GB as reasonable limit)
        memory_score = max(0, 1.0 - (performance_metrics['memory_usage_mb'] / 2048.0))
        efficiency_factors.append(('Memory Efficiency', memory_score, f"{performance_metrics['memory_usage_mb']:.0f} MB"))
    
    if efficiency_factors:
        overall_score = sum(score for _, score, _ in efficiency_factors) / len(efficiency_factors)
        
        print(f"   Efficiency Components:")
        for factor_name, score, display_value in efficiency_factors:
            score_pct = score * 100
            print(f"     {factor_name}: {score_pct:.1f}% ({display_value})")
        
        print(f"\n   Overall Pipeline Efficiency: {overall_score*100:.1f}%")
        
        # Performance recommendations
        if overall_score > 0.8:
            print("   Status: EXCELLENT - Pipeline is performing very well")
        elif overall_score > 0.6:
            print("   Status: GOOD - Pipeline performance is acceptable")
        elif overall_score > 0.4:
            print("   Status: MODERATE - Consider optimization opportunities")
        else:
            print("   Status: NEEDS IMPROVEMENT - Performance optimization recommended")
    
except Exception as e:
    print(f"   Could not calculate efficiency assessment: {e}")

print(f"\nPerformance analysis completed!")
print(f"Pipeline metrics captured at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Force garbage collection to clean up memory
gc.collect()

SECTION 7: PIPELINE PERFORMANCE METRICS
1. Memory Usage Analysis:
   Current Memory Usage: 204.41 MB
   Could not analyze memory usage: name 'spark' is not defined

2. Data Processing Stages Summary:

3. Feature Engineering Impact:

4. Processing Time Estimation:

5. Resource Utilization Summary:
   CPU Usage: 4.8%
   Workspace Disk Usage: 7.7%
   Available Disk Space: 877.7 GB

6. Pipeline Efficiency Assessment:

Performance analysis completed!
Pipeline metrics captured at: 2025-09-29 21:38:16
   CPU Usage: 4.8%
   Workspace Disk Usage: 7.7%
   Available Disk Space: 877.7 GB

6. Pipeline Efficiency Assessment:

Performance analysis completed!
Pipeline metrics captured at: 2025-09-29 21:38:16


25

## Section 8: Interactive Data Exploration

Finally, let's demonstrate how to convert our processed Spark DataFrame to Pandas for interactive analysis and create sample visualizations using our SalaryVisualizer class.

In [11]:
# Interactive Data Exploration and Visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

print("SECTION 8: INTERACTIVE DATA EXPLORATION")
print("=" * 55)

if hasattr(processor, 'df_processed') and processor.df_processed is not None:
    print("1. Spark to Pandas Conversion:")
    
    try:
        # Convert a sample to Pandas for interactive analysis
        sample_size = min(5000, processor.df_processed.count())  # Limit to 5k records for demo
        
        print(f"   Converting {sample_size:,} records to Pandas DataFrame...")
        pandas_df = processor.df_processed.limit(sample_size).toPandas()
        
        print(f"   ✓ Conversion successful!")
        print(f"   Pandas DataFrame Shape: {pandas_df.shape}")
        print(f"   Memory Usage: {pandas_df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")
        
        # Display basic info about the Pandas DataFrame
        print(f"\n2. Pandas DataFrame Overview:")
        print(f"   Data Types:")
        
        # Show column types summary
        type_counts = pandas_df.dtypes.value_counts()
        for dtype, count in type_counts.items():
            print(f"     {dtype}: {count} columns")
        
        # Show sample data
        print(f"\n   Sample Records (first 3 rows):")
        display_columns = ['TITLE', 'COMPANY', 'SALARY_AVG_IMPUTED', 'EXPERIENCE_LEVEL_CLEAN']
        available_display_cols = [col for col in display_columns if col in pandas_df.columns]
        
        if available_display_cols:
            sample_data = pandas_df[available_display_cols].head(3)
            for idx, row in sample_data.iterrows():
                print(f"     Row {idx + 1}:")
                for col in available_display_cols:
                    value = row[col]
                    if pd.isna(value):
                        display_val = "NULL"
                    elif 'SALARY' in col and isinstance(value, (int, float)):
                        display_val = f"${value:,.0f}"
                    else:
                        display_val = str(value)[:50] + ("..." if len(str(value)) > 50 else "")
                    print(f"       {col}: {display_val}")
        
    except Exception as e:
        print(f"   ✗ Pandas conversion failed: {e}")
        pandas_df = None
    
    print(f"\n3. Quick Statistical Analysis:")
    
    if pandas_df is not None:
        try:
            # Basic statistics for numerical columns
            numeric_columns = pandas_df.select_dtypes(include=['number']).columns
            
            if len(numeric_columns) > 0:
                print(f"   Numerical Columns Analysis:")
                
                for col in numeric_columns[:3]:  # Show first 3 numeric columns
                    if pandas_df[col].notna().sum() > 0:
                        stats = pandas_df[col].describe()
                        print(f"     {col}:")
                        print(f"       Mean: {stats['mean']:,.2f}")
                        print(f"       Median: {stats['50%']:,.2f}")
                        print(f"       Range: {stats['min']:,.0f} - {stats['max']:,.0f}")
            
            # Top categories for text columns
            text_columns = pandas_df.select_dtypes(include=['object']).columns
            print(f"\n   Categorical Analysis (top 3 categories):")
            
            for col in text_columns[:2]:  # Show first 2 text columns
                if col in pandas_df.columns:
                    top_values = pandas_df[col].value_counts().head(3)
                    print(f"     {col}:")
                    for value, count in top_values.items():
                        print(f"       {value}: {count:,} ({count/len(pandas_df)*100:.1f}%)")
        
        except Exception as e:
            print(f"   Could not perform statistical analysis: {e}")
    
    print(f"\n4. Sample Visualization Creation:")
    
    # Try to create a simple visualization using our SalaryVisualizer
    try:
        # Initialize SalaryVisualizer with our processed data
        visualizer = SalaryVisualizer()
        
        # Set up basic configuration
        visualizer.df = processor.df_processed
        
        print("   Initializing SalaryVisualizer...")
        print(f"   ✓ Visualizer ready with {processor.df_processed.count():,} records")
        
        # Create a simple chart (experience vs salary if columns exist)
        salary_col = 'SALARY_AVG_IMPUTED' if 'SALARY_AVG_IMPUTED' in processor.df_processed.columns else None
        experience_col = 'EXPERIENCE_LEVEL_CLEAN' if 'EXPERIENCE_LEVEL_CLEAN' in processor.df_processed.columns else None
        
        if salary_col and experience_col and pandas_df is not None:
            print(f"   Creating sample visualization: {experience_col} vs {salary_col}")
            
            # Create a simple matplotlib visualization
            plt.figure(figsize=(10, 6))
            
            # Group data for visualization
            grouped_data = pandas_df.groupby(experience_col)[salary_col].agg(['mean', 'median', 'count']).reset_index()
            
            # Create bar plot
            plt.bar(grouped_data[experience_col], grouped_data['mean'], alpha=0.7, color='skyblue', label='Mean Salary')
            plt.bar(grouped_data[experience_col], grouped_data['median'], alpha=0.7, color='orange', width=0.5, label='Median Salary')
            
            plt.title('Salary Distribution by Experience Level\n(Pipeline Demo Output)', fontsize=14, fontweight='bold')
            plt.xlabel('Experience Level', fontsize=12)
            plt.ylabel('Salary ($)', fontsize=12)
            plt.xticks(rotation=45)
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # Format y-axis as currency
            plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
            
            plt.tight_layout()
            
            # Save the plot
            chart_path = f"/home/samarthya/sourcebox/github.com/project-from-scratch/figures/pipeline_demo_chart_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
            plt.savefig(chart_path, dpi=300, bbox_inches='tight')
            plt.show()
            
            print(f"   ✓ Chart created and saved: {chart_path}")
            
            # Display summary statistics
            print(f"   Chart Summary:")
            for _, row in grouped_data.iterrows():
                exp_level = row[experience_col]
                mean_sal = row['mean']
                count = row['count']
                print(f"     {exp_level}: ${mean_sal:,.0f} avg ({count:,} jobs)")
        
        else:
            print(f"   Required columns not available for visualization")
            print(f"   Available columns: {list(processor.df_processed.columns)[:5]}...")
    
    except Exception as e:
        print(f"   ✗ Visualization creation failed: {e}")
    
    print(f"\n5. Data Export for External Tools:")
    
    if pandas_df is not None:
        try:
            # Export sample to Excel for business users
            excel_path = f"/home/samarthya/sourcebox/github.com/project-from-scratch/data/processed/pipeline_demo_sample_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
            pandas_df.to_excel(excel_path, index=False, engine='openpyxl')
            print(f"   ✓ Excel export: {excel_path}")
            
            # Create summary report
            summary_stats = {
                'total_records': len(pandas_df),
                'unique_companies': pandas_df['COMPANY'].nunique() if 'COMPANY' in pandas_df.columns else 'N/A',
                'avg_salary': pandas_df['SALARY_AVG_IMPUTED'].mean() if 'SALARY_AVG_IMPUTED' in pandas_df.columns else 'N/A',
                'date_generated': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            
            print(f"   Summary Report:")
            for key, value in summary_stats.items():
                if isinstance(value, float):
                    print(f"     {key.replace('_', ' ').title()}: {value:,.2f}")
                else:
                    print(f"     {key.replace('_', ' ').title()}: {value}")
            
        except Exception as e:
            print(f"   Export to Excel failed: {e}")
    
    print(f"\n6. Pipeline Demo Completion Summary:")
    print("   ✓ Raw data successfully loaded using SparkJobAnalyzer")
    print("   ✓ Data cleaning and feature engineering completed with JobMarketDataProcessor")
    print("   ✓ Data quality assessment passed")
    print("   ✓ Multi-format export operations successful")
    print("   ✓ Performance metrics captured")
    print("   ✓ Interactive analysis with Pandas conversion")
    print("   ✓ Sample visualization created")
    print("   ✓ Data ready for production analytics workflows")
    
    print(f"\nDemonstration completed successfully!")
    print(f"Your data processing pipeline is now fully operational and validated.")
    
else:
    print("Cannot perform interactive exploration - processed data not available")

SECTION 8: INTERACTIVE DATA EXPLORATION
Cannot perform interactive exploration - processed data not available
